# KNN Imputation: Terry Stops Dataset

Now we're going to work on seeing how well KNN Imputation works on a dataset that is composed mostly of categorical data.

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
t_stops = pd.read_csv('datasets/terry_stops/eda_clean.csv')
t_stops = t_stops.drop('Unnamed: 0', axis=1)
t_stops.head()

,subject_age_group,subject_id,go_sc_num,terry_stop_id,stop_resolution,weapon_type,officer_id,officer_gender,officer_race,subject_perceived_race,...,officer_age,field_contact,offense_report,dif_race,dif_gender,dif_race_gender,12am_6am,7am_12pm,1pm_6pm,7pm_11pm
0,Unknown,unassigned,20150000110760,33273,Offense Report,None,6355,F,White,White,...,45,n,y,N,N,Y,0,1,0,0
1,Unknown,unassigned,20150000110760,33274,Offense Report,None,6355,F,White,White,...,45,n,y,N,Y,Y,0,1,0,0
2,Unknown,unassigned,20150000120532,35462,Arrest,None,7735,M,White,Not Specified,...,24,n,n,Y,Y,Y,0,0,1,0
3,Unknown,unassigned,20150000127841,36993,Offense Report,None,7474,F,White,Not Specified,...,46,n,y,Y,Y,Y,0,0,1,0
4,Unknown,unassigned,20150000145811,41041,Arrest,None,4835,M,White,Not Specified,...,56,n,n,Y,N,Y,0,0,1,0


# Preprocessing
Need to drop unusable columns such as unique IDs.

In [3]:
t_stops = t_stops.drop(['subject_id', 'go_sc_num', 'terry_stop_id', 
                        'officer_id'], axis=1)
t_stops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30364 entries, 0 to 30363
Data columns (total 28 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   subject_age_group         30364 non-null  object
 1   stop_resolution           30364 non-null  object
 2   weapon_type               30364 non-null  object
 3   officer_gender            30364 non-null  object
 4   officer_race              30364 non-null  object
 5   subject_perceived_race    30364 non-null  object
 6   subject_perceived_gender  30364 non-null  object
 7   initial_call_type         30364 non-null  object
 8   final_call_type           30364 non-null  object
 9   call_type                 30364 non-null  object
 10  arrest_flag               30364 non-null  object
 11  frisk_flag                30364 non-null  object
 12  precinct                  30364 non-null  object
 13  sector                    30364 non-null  object
 14  beat                  

In [4]:
# Create experiment copy
stops_exp = t_stops.copy()